In [1]:
import numpy as np
import os
from mintpy.objects import sensor
from mintpy.utils import ptime, readfile, utils as ut, writefile
from mintpy.objects.stack import timeseries
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
timseries_file =  r'C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries_TRE_GNSS.h5';
output_dir =  r'C:\Users\bvarugu\Documents\timseries2csv';
output_unit = 'mm';
output_prefix = "ARIA_VERT_TS"
os.makedirs(output_dir,exist_ok = True);
chunk_size=1000000;
obj = timeseries(timseries_file);
ts_atr = obj.get_metadata();
ts_dates = obj.get_date_list();
col_names = ["Lat", "Lon"] + [f"D{date}" for date in ts_dates]
print(col_names)

['Lat', 'Lon', 'D20220401', 'D20220501', 'D20220601', 'D20220701', 'D20220801', 'D20220901', 'D20221001', 'D20221101', 'D20221201', 'D20230101', 'D20230201', 'D20230301', 'D20230401', 'D20230501', 'D20230601', 'D20230701', 'D20230801', 'D20230901', 'D20231001', 'D20231101', 'D20231201', 'D20240101', 'D20240201', 'D20240301', 'D20240401', 'D20240501', 'D20240601', 'D20240701', 'D20240801', 'D20240901']


In [3]:
coord = ut.coordinate(ts_atr)
ys, xs = np.indices([int(ts_atr['LENGTH']),int(ts_atr['WIDTH'])]);
lats, lons = coord.radar2geo(ys.flatten(),xs.flatten())[:2];

In [4]:
if output_unit == 'ft':
    conv_factor = 3.28084;
elif output_unit == 'mm':
    conv_factor = 1000;
else:
    conv_factor = 1.0;
data_series = [];
for i,date in enumerate(ts_dates):
    data = readfile.read(timseries_file, datasetName=date)[0];
    data *= conv_factor;data = np.round(data,1);
    data_series.append(data.flatten())
data_series = np.column_stack(data_series)    

In [5]:
coords = np.column_stack([np.array(lats).flatten(), np.array(lons).flatten()]);
coords = coords.astype(np.float32);
valid_mask = ~np.all(np.isnan(data_series), axis=1)
coords = coords[valid_mask]
data_series = data_series[valid_mask];data_series = data_series.astype(np.float32);
data_series = np.where(np.isnan(data_series), "<Null>", data_series);


In [6]:

num_points = data_series.shape[0];
num_chunks = int(np.ceil(num_points / chunk_size));
print(num_chunks)

16


In [7]:

# for chunk_idx in range(num_chunks):
#     start = chunk_idx * chunk_size
#     end = min((chunk_idx + 1) * chunk_size, num_points)
#     chunk_data = data_series[start:end];
#     chunk_coords = coords[start:end];
#     combined_data = np.column_stack([chunk_coords, chunk_data]);

#     # Create a DataFrame for the chunk
#     chunk_df = pd.DataFrame(combined_data, columns=col_names)

#     # Save the chunk to a CSV file
#     output_csv = f"{output_prefix}_{chunk_idx + 1}.csv";
#     output_csv = os.path.join(output_dir,output_csv)
#     chunk_df.to_csv(output_csv, index=False)
#     print(f"Saved chunk {chunk_idx + 1} to {output_csv}")

In [8]:
combined_data_stack = np.column_stack([coords, data_series]);

In [9]:
df = pd.DataFrame(combined_data_stack, columns=col_names);
output_csv = f"{output_prefix}_full_data_{output_unit}.csv";
output_csv = os.path.join(output_dir,output_csv)
df.to_csv(output_csv, index=False)
print(f"Saved chunk to {output_csv}")

Saved chunk to C:\Users\bvarugu\Documents\timseries2csv\ARIA_VERT_TS_full_data_ft.csv
